In [1]:
import os
import sys
import dotenv
from pathlib import Path
from IPython.display import Markdown as md

sys.path.append(os.path.expanduser("~/source/appland/SWE-bench"))

project_dir = Path(os.path.expanduser("~")) / "source" / "appland" / "appmap-js"
# Expand home path to full path
os.chdir(project_dir)

from appmap.navie.editor import Editor
from appmap.navie.format_instructions import xml_format_instructions
from appmap.navie.fences import extract_fenced_content

dotenv.load_dotenv(".env.notebook", override=True)
plan_dir = project_dir / "notebooks" / "prompt_file_changes_xml"

In [4]:
import re


prompt_files_raw = Editor(str(plan_dir / "prompt_files")).ask(
    f"""@generate

List all files that contain Navie prompts.
""", prompt="""
Emit as a plain text list with one file per line.
Do not include any code fences or other explanations.
""")

prompt_files = prompt_files_raw.split("\n")
# Strip leading '-' char, if any, and whitespace
prompt_files = [prompt_file.lstrip("-").strip() for prompt_file in prompt_files]

print(f"Planning changes to: {"  ".join(prompt_files)}")

prompts = {}
for prompt_file in prompt_files:
    # Replace problematic characters in the file name using Regexp
    sanitized_file_name = re.sub(r"[^a-zA-Z0-9]", "_", prompt_file)
    with open(prompt_file) as f:
        content = f.read()
    prompts_raw = Editor(str(plan_dir / "prompt_file" / sanitized_file_name)).ask(
        f"""@explain /nocontext

Extract all Navie prompts from the context file.
If there are no prompts, write "No prompts found."

Response format:

<prompt>
  <name>prompt_name</name>
  <content>
    prompt_content
  </content>
</prompt>

... for each prompt in the file.

Input:

<context>
{content}
</context>
"""
    )

    # Extract prompt tags by regexp
    prompt_tags = re.findall(r"<prompt>.*?</prompt>", prompts_raw, re.DOTALL)
    prompts[prompt_file] = []
    for prompt_tag in prompt_tags:
        prompt_name = re.search(r"<name>(.*?)</name>", prompt_tag).group(1)
        prompt_content = re.search(r"<content>(.*?)</content>", prompt_tag, re.DOTALL).group(1)
        prompts[prompt_file].append((prompt_name, prompt_content))

print(prompts)

  Using cached answer
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/prompt_file_changes_xml/prompt_files/ask/ask.md
Planning changes to: packages/navie/src/agents/explain-agent.ts  packages/navie/src/agents/test-agent.ts  packages/navie/src/agents/plan-agent.ts  packages/navie/src/agents/generate-agent.ts  packages/navie/src/agents/help-agent.ts  packages/navie/src/prompt.ts
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/prompt_file_changes_xml/prompt_file/packages_navie_src_agents_explain_agent_ts/ask/ask.md
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/prompt_file_changes_xml/prompt_file/packages_navie_src_agents_test_agent_ts/ask/ask.md
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/prompt_file_changes_xml/prompt_file/packages_navie_src_agents_plan_agent_ts/ask/ask.md
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/prompt_file_changes_xml/promp

In [12]:

# Select the first two items in 'prompts'
# two_prompts = list(prompts.items())[:2]
for prompt_file, prompt_list in prompts.items():
    with open(prompt_file) as f:
        content = f.read()
    for prompt_name, prompt_content in prompt_list:
        updated_prompt = Editor(
            str(plan_dir / "prompt_update" / f"{prompt_name}.xml")
        ).ask(
            f"""@explain /nocontext

You're provided with OLD-PROMPT, the content of an LLM prompt.

Your task is to modify the OLD-PROMPT to require the LLM to output code 
suggestions in the following format:

<code>
    <language>language name</language>
    <file>path/to/file</file>
    <content>
        ( Updated code content )
    </content>
</code>

Output just the NEW-PROMPT, and nothing else. Don't wrap the NEW-PROMPT in tags.

---OLD-PROMPT---
{prompt_content}
---NEW-PROMPT---
"""
        )
        print(updated_prompt)

  Using cached answer
  Output is available at /Users/kgilpin/source/appland/appmap-js/notebooks/prompt_file_changes_xml/prompt_update/EXPLAIN_AGENT_PROMPT.xml/ask/ask.md
**Task: Explaining Code, Analyzing Code, Generating Code**

## About you

Your name is Navie. You are an AI software architect created and maintained by AppMap Inc, and are available to
AppMap users as a service.

## About the user

The user is a software developer who is working to understand, maintain and improve a codebase. You can
expect the user to be proficient in software development.

You do not need to explain the importance of programming concepts like planning and testing, as the user is 
already aware of these.

## Your response

1. **Markdown**: Respond using Markdown, unless told by the user to use a different format.

2. **File Paths**: Include paths to source files that are relevant to the explanation.

3. **Length**: Keep your response concise and to the point. If the user asks for code generation, fo